# PSTAT134 Final Project
## Predicting NBA Player Positions

In [ ]:
import pandas as pd
import numpy as np
import sklearn.decomposition as skld
import matplotlib.pyplot as plt
import seaborn as sns
import csv

import helper_basketball as h
import imp
imp.reload(h);

In [ ]:
#import data
agg_DF = pd.read_csv('NBAPlayerData.csv')

agg_DF

#rename one column
agg_DF.rename(columns = {'\ufeffPlayer': 'Player'}, inplace = True)

In [ ]:
def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    
    from pandas import DataFrame 
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(DataFrame(d, columns=h))

In [ ]:
def get_nba_dash(params, return_url=False):
    
    from pandas import DataFrame 
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/leaguedashplayerstats" + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(DataFrame(d, columns=h))

In [ ]:
#get all players
params = {'LeagueID':'00', 
          'Season': '2016-17', 
          'IsOnlyCurrentSeason': '0'}
players = get_nba_data('commonallplayers', params)
players = players[players.TO_YEAR == '2017']
players.head()

In [ ]:
#get specific players game stats
params = {'PlayerID':'1627826',
          'Season':'2016-17',
          'SeasonType':'Regular Season'}

gamedata = get_nba_data('playergamelog', params)
gamedata.head()

In [ ]:
#get specific players common info
player_params = {'PlayerID':'2544'}

playerdata = get_nba_data('commonplayerinfo', player_params)
playerdata.head()

In [ ]:
#get specific ratings per player
dash_params = {"MeasureType":"Advanced",
               "PerMode":"PerGame",
               "PlusMinus":"N",
               "PaceAdjust":"N",
               "Rank":"N",
               "LeagueID":"00",
               "Season":"2016-17",
               "SeasonType":"Playoffs",
               "PORound":'0',
               "Outcome":'',
               "Location":'',
               "Month":'0',
               "SeasonSegment":'',
               "DateFrom":'',
               "DateTo":'',
               "OpponentTeamID":'0',
               "VsConference":'',
               "VsDivision":'',
               "TeamID":'0',
               "Conference":'',
               "Division":'',
               "GameSegment":'',
               "Period":'0',
               "ShotClockRange":'',
               "LastNGames":'0',
               "GameScope":'',
               "PlayerExperience":'',
               "PlayerPosition":'',
               "StarterBench":'',
               "DraftYear":'',
               "DraftPick":'',
               "College":'',
               "Country":'',
               "Height":'',
               "Weight":''}
ratingsdata = get_nba_dash(dash_params)
ratingsdata.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
df = pd.read_csv('NBAPlayerData.csv')

# using the statistics to predict Position
training_features = ['PER', 'TS', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'USG',
       'ORtg', 'DRtg', 'OWS', 'DWS', 'WS']
# training_features = training_features.reshape(1,-1)
target = 'Position'

# splitting into training and test set 
train_x, test_x, train_y, test_y = train_test_split(df[training_features], df[target], train_size = 0.7, test_size = 0.3)
# reshaping the test data
test_x = np.array(test_x).reshape(1,-1)

# fitting the Logistic Regression model
logistic = LogisticRegression()
model = logistic.fit(train_x, train_y)

In [ ]:
model.score(train_x, train_y)

In [ ]:
# plot for the distribution of labels in the data
import seaborn as sns

df = pd.read_csv('NBAPlayerData.csv')
# print(df.shape)



sns.countplot(x="Position". data=df)